### Optimal pooling example

Some characteristics of optimal pooling:
- **Different bins**: Bins should be distinct from each other and not overlap. This ensures that the continuous variable is properly discretized and can help avoid confusion when interpreting the results.
- **Balanced distribution**: The number of observations in each bin should be roughly equal or at least not highly skewed. This helps ensure that each bin contributes meaningfully to the model and prevents any single bin from dominating the results.
- **Minimal Gini decrease**: The Gini index is a measure of statistical dispersion, often used to assess the quality of a split in decision trees. In optimal binning, a smaller decrease in the Gini index indicates that the bins are more homogeneous, leading to better model performance.
- **Monotonic relationship**: Ideally, bins should have a monotonic relationship with the target variable, meaning that the relationship should consistently increase or decrease as the bin values change. This simplifies interpretation and can improve model performance.
- **Interpretability**: The bins should be easy to understand and interpret, with meaningful boundaries that are relevant to the domain of the problem. This helps stakeholders better understand the results and can aid in decision-making.
- **Stability**: Good optimal binning should be stable and not sensitive to small changes in the data. This ensures that the model's performance is robust and reliable.
- **Domain knowledge**: The binning process should take into account domain knowledge and expert input when determining the bin boundaries. This can help ensure that the bins are meaningful and relevant to the problem at hand.

In [1]:
from ortools.sat.python import cp_model
from scipy import special
from sklearn.tree import DecisionTreeRegressor, _tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

### A. Create a synthetic dataset

In [2]:
# Set a seed so the randomness is reproducible
np.random.seed(0)

# Number of samples
n_samples = 100000

# Generate columns
target = np.random.binomial(1, 0.05, n_samples)
year = np.random.randint(2010, 2023, n_samples)

x1 = np.random.rand(n_samples)
x1_noise = np.random.normal(0, 0.5, n_samples)
x1 = 0.5 * x1 + 0.5 * target + x1_noise

x2 = np.sin(3*np.pi*np.random.rand(n_samples))
x2_noise = np.random.normal(0, 0.5, n_samples)
x2 = 0.5 * x2 + 0.5 * target + x2_noise

x3_noise = np.random.normal(0, 0.5, n_samples)
x3 = 0.5 * x1 + 0.1 * target + x1_noise

# Consolidate all the columns into a DataFrame
df = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'year': year,
    'target': target,
})

# Define the logistic regression model
model = LogisticRegression()

# Fit the model
X = df[['x2', 'x3']]
y = df['target']
model.fit(X, y)

# Make predictions (probabilities)
y_pred_proba = model.predict_proba(X)[:, 1]
df['score'] = y_pred_proba

# Compute Gini coefficient
gini = 2*roc_auc_score(y, y_pred_proba) - 1
print(f"Gini: {gini:.2%}")

Gini: 49.57%


### B. Create possible cut-off thresholds and annual default rates

This step can be replaced by equal-width / equal-height / xgboost binning

In [3]:
# Set the number of bins you want
n_bins = 30

# Set the max_depth, min_samples_leaf, and max_leaf_nodes parameters
clf = DecisionTreeRegressor(max_depth=n_bins, 
                            min_samples_leaf=1000, 
                            max_leaf_nodes=n_bins)

clf.fit(df[['score']], df['target'])
thresholds = clf.tree_.threshold[clf.tree_.threshold > _tree.TREE_UNDEFINED]

# Add minimum and maximum edges
bins = sorted([-np.inf] + list(thresholds) + [np.inf])
df['bins'] = pd.cut(df['score'], bins, labels=False)

print([round(num, 4) for num in bins])

[-inf, 0.0113, 0.0187, 0.0278, 0.0285, 0.0299, 0.0309, 0.0375, 0.0411, 0.0426, 0.0536, 0.0557, 0.0574, 0.0628, 0.0647, 0.0669, 0.0702, 0.0741, 0.0766, 0.08, 0.0837, 0.0988, 0.1057, 0.1133, 0.1203, 0.1288, 0.142, 0.1578, 0.1877, 0.2228, inf]


In [4]:
drs = df.pivot_table(index='year', columns='bins', values='target', aggfunc='mean', margins='all').T
drs.style.format("{:.2%}")

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,All
bins,,,,,,,,,,,,,,
0,0.91%,0.91%,0.86%,0.56%,0.28%,0.86%,0.82%,0.74%,0.47%,0.85%,0.47%,0.82%,0.77%,0.72%
1,1.78%,1.19%,0.66%,2.06%,0.91%,1.76%,1.25%,1.09%,2.03%,2.05%,1.86%,2.07%,1.17%,1.53%
2,2.54%,2.45%,2.29%,2.46%,2.61%,2.88%,2.68%,3.12%,2.29%,2.67%,2.59%,2.64%,1.66%,2.53%
3,2.99%,6.06%,1.54%,5.88%,3.17%,1.39%,7.95%,6.58%,1.18%,5.88%,5.88%,3.23%,3.41%,4.30%
4,1.91%,1.14%,4.55%,2.03%,1.55%,5.16%,3.57%,2.50%,1.39%,1.42%,0.62%,1.21%,1.89%,2.21%
5,5.60%,5.00%,2.52%,5.36%,0.87%,7.69%,2.94%,2.17%,3.15%,3.96%,6.60%,3.81%,5.26%,4.12%
6,2.25%,3.69%,3.85%,3.99%,2.12%,2.74%,3.43%,3.23%,3.24%,2.47%,3.27%,4.21%,2.16%,3.12%
7,5.49%,7.04%,4.98%,3.61%,5.14%,5.19%,4.42%,4.15%,4.26%,3.15%,3.40%,4.91%,5.73%,4.69%
8,3.00%,1.94%,0.89%,4.08%,3.36%,0.00%,3.70%,3.60%,4.46%,5.41%,3.74%,4.20%,2.86%,3.19%


### C. Create optimisation function
Some functions are inspired by optbinning.

In [5]:
def calculate_iv(events, non_events, total_events, total_non_events):
    # Calculate the percentage of events and non-events
    event_rate = events / total_events
    non_event_rate = non_events / total_non_events

    # Add a constant to the denominator to avoid division by zero
    event_rate = np.where(event_rate == 0, 0.000001, event_rate)

    WoE = np.log(non_event_rate / event_rate)
    IV = (non_event_rate - event_rate) * WoE

    return IV

def create_consecutive_bin_iv(events, counts):
    n = len(events)
    iv_dict = {}
    total_events = events.sum()
    total_non_events = (counts - events).sum()

    for i in range(n+1):
        for j in range(i, n):
            # Combine events and non-events for each combination
            combined_events = events[i:j+1].sum()
            combined_non_events = (counts[i:j+1] - events[i:j+1]).sum()

            # Calculate IV for this bin combination
            iv_dict[(i+1, j+1)] = calculate_iv(combined_events, 
                                               combined_non_events, 
                                               total_events, 
                                               total_non_events)

    return iv_dict

def test_proportions(event_1, non_event_1, event_2, non_event_2):
    total_1 = event_1 + non_event_1
    total_2 = event_2 + non_event_2
    proportion_1 = event_1 / total_1
    proportion_2 = event_2 / total_2
    pooled_proportion = (event_1 + event_2) / (total_1 + total_2)

    z = ((proportion_1 - proportion_2) / np.sqrt(pooled_proportion * 
         (1 - pooled_proportion) * (1 / total_1 + 1 / total_2)))
    return abs(z)              
                
def individual_bin_stats(events, counts):
    ''' Create stats for all purposes'''
    n = len(events)
    stats = {}
    total_events = events.sum()
    total_non_events = (counts - events).sum()

    for i in range(n+1):
        for j in range(i, n):
            # Calculate the combined events and non-events for each bin combination
            combined_events = events[i:j+1].sum()
            combined_non_events = (counts[i:j+1] - events[i:j+1]).sum()

            # Calculate IV for this bin combination
            iv = calculate_iv(combined_events, combined_non_events, total_events, total_non_events)
            default_rate = combined_events / (combined_events + combined_non_events)
                
            stats[(i+1, j+1)] = [iv, 
                                 combined_events, 
                                 combined_non_events, 
                                 total_events, 
                                 total_non_events, 
                                 default_rate]

    return stats

def bin_combo_stats(stats_b):
    result_stats = {}
    for a in stats_b:
        for b in stats_b:
            if a[1] == b[0]-1:
                e1 = stats_b[a][1]
                ne1 = stats_b[a][2]
                e2 = stats_b[b][1]
                ne2 = stats_b[b][2]
                
                # Calculate mononicity of consecutive default rates
                default_rate_1 = stats_b[a][5]
                default_rate_2 = stats_b[b][5]
                trend = 0 if default_rate_1 > default_rate_2 else 1
                
                # CaLculate z-score of adjacent bins
                z_score = test_proportions(e1, ne1, e2, ne2)
                
                # Store results
                result_stats[a, b] = [z_score, trend]
                
    return result_stats
                
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """ Intermediate solutions for debugging and multiple solutions """

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            pass
            

def create_pools(events, non_events, lra_drs, min_bins, max_bins):
    ''' Finds optimal binning given potential cutoffs '''
    
    n = len(events)
    
    # Create the model.
    model = cp_model.CpModel()

    # Variables
    x = {}
    iv = {}
    bins = []
    for i in range(1, n+1):
        for j in range(i, n+1):
            # x[i, j] is a boolean that is true if included in the combination
            x[i, j] = model.NewBoolVar(f'x[{i},{j}]')
    
    # Create supporting data
    bin_data = individual_bin_stats(events, non_events)        
    iv = {key: val[0] for key, val in bin_data.items()}
    
    # Testing: todo delete
    # iv[2, 4] = 12007
    
    # Create bin options that should be excluded from consideration
    excluded_bin_combos = bin_combo_stats(bin_data)
    
    # Testing: todo delete
    # excluded_bin_combos[(2, 4), (5, 6)] = [0.1, 1]
    # excluded_bin_combos[(2, 4), (5, 5)] = [0.1, 0]
    
    # Constraints
    for i in range(1, n+1):
        # Each bin i should be included in exactly one tuple.
        model.Add(sum(x[j, k] for j in range(1, i+1) for k in range(i, n+1)) == 1)
        
    # Reject bins if the z-score is below the desired threshold (todo: check signs)
    for key, value in excluded_bin_combos.items():
        if value[0] < 1.0:
            ((i1, j1), (i2, j2)) = key
            model.Add(x[i1, j1] + x[i2, j2] <= 1)
            
    # Reject bins if the default rates are not monotonically increasing
    # for key, value in excluded_bin_combos.items():
    #     if value[1] == 1:
    #         ((i1, j1), (i2, j2)) = key
    #         model.Add(x[i1, j1] + x[i2, j2] <= 1)    

    # Maximum number of bins (max_bins)
    model.Add(sum(x[i, j] for i in range(1, n+1) for j in range(i, n+1)) <= max_bins)

    # Minimum number of bins (min_bins)
    model.Add(sum(x[i, j] for i in range(1, n+1) for j in range(i, n+1)) >= min_bins)


    # Objective function
    model.Maximize(sum(iv[i, j] * x[i, j] for i in range(1, n+1) for j in range(i, n+1)))

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(x)
    status = solver.SolveWithSolutionCallback(model, solution_printer)

    if status == cp_model.OPTIMAL:
        print('Total IV =', solver.ObjectiveValue(), cp_model.OPTIMAL, cp_model.FEASIBLE)
        for i in range(1, n+1):
            for j in range(i, n+1):
                if solver.Value(x[i, j]) == 1:
                    print(i, j)
                    bins.append([i, j])

    return bins, status, solver, iv, bin_data, excluded_bin_combos

### D. Solve the optimisation task

In [6]:
# Get input data per bin
stats_combined = df.groupby('bins').agg({'target': ['sum', 'count']})
stats_combined = stats_combined.reset_index()
stats_combined.columns = ['bins', 'events', 'count']
stats_combined['non_events'] = stats_combined['count'] - stats_combined['events']
stats_combined['default_rate'] = stats_combined['events'] / stats_combined['count']

# Add long-run averages
annual_averages = df.groupby(['bins', 'year'])['score'].mean()
average_scores_per_bin = annual_averages.groupby('bins').mean().reset_index()
average_scores_per_bin.columns = ['bins', 'lra_dr']

stats_combined = pd.merge(stats_combined, average_scores_per_bin, on='bins', how='left')

stats_combined

,bins,events,count,non_events,default_rate,lra_dr
0,0,98,13694,13596,0.007156,0.007404
1,1,217,14158,13941,0.015327,0.014954
2,2,368,14533,14165,0.025322,0.023071
3,3,43,1001,958,0.042957,0.028124
4,4,44,1995,1951,0.022055,0.029161
5,5,60,1455,1395,0.041237,0.030403
6,6,250,8000,7750,0.031250,0.034142
7,7,174,3708,3534,0.046926,0.039283
8,8,45,1412,1367,0.031870,0.041812
9,9,434,8969,8535,0.048389,0.047746


In [7]:
# Test solution
events = stats_combined['events']
non_events = stats_combined['non_events']
lra_drs = stats_combined['lra_dr']

bins, _, _, iv, bin_data, _ = create_pools(events, non_events, lra_drs, 1, 5)

Total IV = 1.0012383855569686 4 2
1 2
3 9
10 24
25 29
30 30


In [8]:
# Create new bins
new_bins = [edge for sublist in bins for edge in sublist]
new_bins[-1] += 1  # To include the last edge in the bin

def map_bins(row, bin_edges):
    for i in range(0, len(bin_edges)-1):
        if bin_edges[i] <= row['bins'] < bin_edges[i+1]:
            return i // 2
    return (len(bin_edges) - 2) // 2

df['new_bins'] = df.apply(map_bins, bin_edges=new_bins, axis=1)

# Show stats
df.groupby('new_bins').agg({'bins':['min', 'max']})

bins    
          min max
new_bins         
0           1   2
1           3   9
2          10  24
3          25  29
4           0   0

In [9]:
bins

[[1, 2], [3, 9], [10, 24], [25, 29], [30, 30]]